In [53]:
import pandas as pd, numpy as np
from ortools.sat.python import cp_model

## Constrained Optimization

### inputs

In [54]:
nodes = pd.read_excel('route_inputs.xlsx', sheet_name='nodes')
paths = pd.read_excel('route_inputs.xlsx', sheet_name='paths')
n_nodes = len(nodes)
n_paths = len(paths)

In [55]:
nodes

,node,description
0,1,origin
1,2,middle point
2,3,middle point
3,4,middle point
4,5,middle point
5,6,middle point
6,7,destination


In [56]:
paths

,node_from,node_to,distance
0,1,2,220
1,1,3,1500
2,2,4,650
3,2,5,900
4,4,7,500
5,5,7,400
6,3,6,500
7,6,7,400


### Initialinzing Model

In [57]:
model = cp_model.CpModel()


### Variables (What do we need to find)?

- p is a path index here and its a binary ; whether we are going to take that path or not
- whether that particular row in path will be activated or not?
- this is a variable we need to look at 

In [58]:
x = np.zeros(n_paths).tolist()
print(x)
for p in paths.index:
    print(p)
    x[p] = model.NewIntVar(0,1,'x[{}]'.format([p]))
print(x[2])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
1
2
3
4
5
6
7
x[[2]]


### objective function (How do we find the best possible outcome)?

- we need to make sure the path we chooses x[p] (1 or 0) has the minimum distance to reach from origin to destination


In [59]:
objFun = sum([x[p] * paths.distance[p] for p in paths.index])
model.Minimize(objFun)

In [60]:
objFun

Sum(Sum(Sum(Sum(Sum(Sum(Sum(ProductCst(x[[0]](0..1), 220), ProductCst(x[[1]](0..1), 1500)), ProductCst(x[[2]](0..1), 650)), ProductCst(x[[3]](0..1), 900)), ProductCst(x[[4]](0..1), 500)), ProductCst(x[[5]](0..1), 400)), ProductCst(x[[6]](0..1), 500)), ProductCst(x[[7]](0..1), 400))

In [61]:
print(objFun)

((((((((220 * x[[0]]) + (1500 * x[[1]])) + (650 * x[[2]])) + (900 * x[[3]])) + (500 * x[[4]])) + (400 * x[[5]])) + (500 * x[[6]])) + (400 * x[[7]]))


### Constrains (What are the conditions we need to keep in mind)?

#### Identifying Orign and Destination 

In [62]:
node_origin = int(nodes.node[nodes['description']=='origin'])
node_destination = int(nodes.node[nodes['description']=='destination'])

print(node_origin)
print(node_destination)

1
7


In [63]:
paths.index[paths.node_from==node_origin]

Int64Index([0, 1], dtype='int64')

In [64]:
paths.index[paths.node_to==node_destination]

Int64Index([4, 5, 7], dtype='int64')

#### Adding constrain : constraint sum(x) == 1 (origin and destination)

- Origin Out is Node in Index 0,1 in paths table 
       - That means either of (1,2) or (1,3) should be active not both
- Destination in Node is at Index 4,5,7 in paths table
       - That means either of (4,7) , (5,7) , (6,7) should be active not all only one

In [65]:
model.Add(sum([x[p] for p in paths.index[paths.node_from==node_origin]]) == 1)
model.Add(sum([x[p] for p in paths.index[paths.node_to==node_destination]]) == 1)

#### Adding constrain : constraint sum(x, in) == sum(x, out)

In [66]:
middle_points = nodes.query("description=='middle point'")
middle_points

,node,description
1,2,middle point
2,3,middle point
3,4,middle point
4,5,middle point
5,6,middle point


In [67]:
for middle_node in middle_points.node:
   sum_in = sum(x[p] for p in paths.index[paths.node_to==middle_node])
   sum_out =sum(x[p] for p in paths.index[paths.node_from==middle_node])
   model.Add(sum_in == sum_out)

### Solveing using Solver

In [68]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

### Results

In [69]:
#print
print('status =', solver.StatusName(status))
print('OF =', solver.ObjectiveValue())

paths['activated'] = 0
for p in paths.index:
    paths.activated[p] = solver.Value(x[p]) ## this prints if the column was activated or not
print(paths)

status = OPTIMAL
OF = 1369.9999999999998
   node_from  node_to  distance  activated
0          1        2       220          1
1          1        3      1500          0
2          2        4       650          1
3          2        5       900          0
4          4        7       500          1
5          5        7       400          0
6          3        6       500          0
7          6        7       400          0
